# 📘 Visualización Interactiva y Experimentos
## Minimización de \(f(x,y)=\dfrac{\arctan(x^2+y^2)}{1+x^2}\)
**Autor:** Agustín A. Carbajal Romero — Grupo C-312

Este notebook permite:
- explorar la forma de la función,
- ejecutar y comparar Máximo Descenso con Armijo y Newton Regularizado,
- combinar ambos (Gradiente → Newton),
- animar la trayectoria iterativa y visualizar la convergencia.

Referencias (conferencias usadas):
- /mnt/data/Conferencia 5 Algoritmos para problemas irrestrictos.pdf  
- /mnt/data/Conferencia 6 Algoritmos para problemas con restricciones.pdf

In [11]:
# Requerido: ejecutar una vez
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import norm, inv
from IPython.display import display, clear_output
import ipywidgets as widgets
import math
plt.rcParams.update({"figure.max_open_warning": 0})

In [12]:
# Función objetivo
def f_xy(x):
    x1, x2 = x[0], x[1]
    return math.atan(x1**2 + x2**2) / (1 + x1**2)

# vectorizado para trabajos con mallas
def f_vec(X, Y):
    return np.arctan(X**2 + Y**2) / (1 + X**2)

# Gradiente analítico (vector)
def grad_f(x):
    x1, x2 = float(x[0]), float(x[1])
    r2 = x1**2 + x2**2
    denom = (1 + x1**2)
    # derivadas (consistentes con la analítica)
    df_dx = (2*x1 * ((1 + x1**2)/(1 + r2**2) - math.atan(r2))) / (denom*denom)
    df_dy = (2*x2) / (denom * (1 + r2**2))
    return np.array([df_dx, df_dy], dtype=float)

# Hessiano numérico (central)
def hessiana_num(f, x, eps=1e-5):
    n = 2
    H = np.zeros((n, n))
    fx = f(x)
    for i in range(n):
        for j in range(n):
            ei = np.zeros(n); ej = np.zeros(n)
            ei[i] = eps; ej[j] = eps
            fpp = f(x + ei + ej)
            fpm = f(x + ei - ej)
            fmp = f(x - ei + ej)
            fmm = f(x - ei - ej)
            H[i,j] = (fpp - fpm - fmp + fmm) / (4 * eps * eps)
    return H

In [13]:
# Gradiente con Armijo (devuelve historia)
def gradiente_armijo_hist(x0, tol=1e-6, maxiter=2000, alpha=1.0, beta=0.5, sigma=1e-4):
    x = np.array(x0, dtype=float)
    hist = [x.copy()]
    g = grad_f(x)
    k = 0
    while norm(g) > tol and k < maxiter:
        d = -g
        t = alpha
        # backtracking Armijo
        while f_xy(x + t*d) > f_xy(x) + sigma * t * np.dot(g, d):
            t *= beta
            if t < 1e-16:
                break
        x = x + t * d
        hist.append(x.copy())
        g = grad_f(x)
        k += 1
    return np.array(hist)

# Newton regularizado (historia)
def newton_reg_hist(x0, tol=1e-6, maxiter=200, lam=1e-6):
    x = np.array(x0, dtype=float)
    hist = [x.copy()]
    g = grad_f(x)
    k = 0
    while norm(g) > tol and k < maxiter:
        H = hessiana_num(f_xy, x)
        # regularizar para garantizar PD
        try:
            eigs = np.linalg.eigvalsh(H)
            min_eig = eigs.min()
            if min_eig <= 0:
                H = H + (abs(min_eig) + lam) * np.eye(2)
            else:
                H = H + lam * np.eye(2)
            dx = -np.linalg.solve(H, g)
        except np.linalg.LinAlgError:
            # fallback a gradiente
            dx = -g
        x = x + dx
        hist.append(x.copy())
        g = grad_f(x)
        k += 1
    return np.array(hist)

# Estrategia combinada: gradiente hasta tol_switch, luego Newton
def combinado_hist(x0, tol_grad=1e-3, tol_newton=1e-8, maxiter_grad=2000, maxiter_newton=200):
    h1 = gradiente_armijo_hist(x0, tol=tol_grad, maxiter=maxiter_grad)
    x_switch = h1[-1]
    h2 = newton_reg_hist(x_switch, tol=tol_newton, maxiter=maxiter_newton)
    # concatenar sin duplicar punto de switch
    return np.vstack((h1, h2[1:]))

In [14]:
# Widgets
x1_w = widgets.FloatText(value=2.0, description='x1:')
x2_w = widgets.FloatText(value=3.0, description='x2:')
method_w = widgets.Dropdown(options=['Gradiente (Armijo)', 'Newton regularizado', 'Combinado'],
                            value='Combinado', description='Método:')
rango_w = widgets.FloatSlider(value=3.0, min=1.0, max=20.0, step=0.5, description='Rango:')
puntos_w = widgets.IntSlider(value=300, min=100, max=600, step=50, description='Puntos:')
tol_grad_w = widgets.FloatLogSlider(value=1e-3, base=10, min=-8, max=-1, step=0.1, description='tol grad:')
tol_newton_w = widgets.FloatLogSlider(value=1e-8, base=10, min=-12, max=-3, step=0.1, description='tol newt:')
run_button = widgets.Button(description='Run', button_style='success')
iter_slider = widgets.IntSlider(value=0, min=0, max=1, step=1, description='Iter:')
output = widgets.Output(layout={'border': '1px solid black'})

ui_left = widgets.VBox([x1_w, x2_w, method_w, widgets.HBox([rango_w, puntos_w])])
ui_right = widgets.VBox([tol_grad_w, tol_newton_w, run_button])
ui = widgets.HBox([ui_left, ui_right])
display(ui, iter_slider, output)

IntSlider(value=0, description='Iter:', max=1)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [17]:
# Variables para almacenar historia actual
current_hist = None

def plot_all(hist, rango, puntos, iter_index=None):
    # hist: np.array of shape (steps,2)
    X = np.linspace(-rango, rango, puntos)
    Y = np.linspace(-rango, rango, puntos)
    Xg, Yg = np.meshgrid(X, Y)
    Z = f_vec(Xg, Yg)
    
    fig = plt.figure(figsize=(14,6))
    
    # Superficie 3D
    ax1 = fig.add_subplot(1,3,1, projection='3d')
    ax1.plot_surface(Xg, Yg, Z, cmap='viridis', edgecolor='none', alpha=0.9)
    ax1.set_title('Superficie 3D')
    ax1.set_xlabel('x'); ax1.set_ylabel('y'); ax1.set_zlabel('f(x,y)')
    # marcar trayectoria completa
    ax1.plot(hist[:,0], hist[:,1], [f_xy(p) for p in hist], 'r.-', label='trayectoria')
    ax1.scatter(0,0,f_xy([0,0]), c='k', marker='*', s=80)
    ax1.legend()
    
    # Contorno + trayectoria
    ax2 = fig.add_subplot(1,3,2)
    cs = ax2.contour(Xg, Yg, Z, levels=30, cmap='viridis')
    ax2.clabel(cs, inline=1, fontsize=8)
    ax2.set_title('Contorno + Trayectoria')
    ax2.set_xlabel('x'); ax2.set_ylabel('y')
    # full traj
    ax2.plot(hist[:,0], hist[:,1], 'r.-')
    ax2.scatter(0,0, c='k', marker='*', s=80)
    if iter_index is not None:
        k = min(iter_index, len(hist)-1)
        ax2.plot(hist[:k+1,0], hist[:k+1,1], 'gold', linewidth=3)
        ax2.scatter(hist[k,0], hist[k,1], c='gold', s=50)
    
    # Heatmap
    ax3 = fig.add_subplot(1,3,3)
    im = ax3.imshow(Z, extent=[-rango,rango,-rango,rango], origin='lower', cmap='viridis', aspect='auto')
    fig.colorbar(im, ax=ax3, shrink=0.8)
    ax3.set_title('Heatmap')
    ax3.set_xlabel('x'); ax3.set_ylabel('y')
    # mark trajectory on heatmap
    ax3.plot(hist[:,0], hist[:,1], 'r.-', markersize=3)
    if iter_index is not None:
        k = min(iter_index, len(hist)-1)
        ax3.scatter(hist[k,0], hist[k,1], c='gold', s=50)
    
    plt.tight_layout()
    plt.show()

def on_run_clicked(b):
    global current_hist
    with output:
        clear_output(wait=True)
        x0 = np.array([x1_w.value, x2_w.value], dtype=float)
        method = method_w.value
        rango = rango_w.value
        puntos = puntos_w.value
        tol_grad = tol_grad_w.value
        tol_newt = tol_newton_w.value
        
        if method == 'Gradiente (Armijo)':
            hist = gradiente_armijo_hist(x0, tol=tol_grad)
        elif method == 'Newton regularizado':
            hist = newton_reg_hist(x0, tol=tol_newt)
        else:
            hist = combinado_hist(x0, tol_grad, tol_newt)
        
        current_hist = hist
        # actualizar slider de iteraciones
        iter_slider.max = max(1, len(hist)-1)
        iter_slider.value = 0
        print(f"Método: {method}")
        print(f"Iteraciones calculadas: {len(hist)-1}")
        print(f"Valor final f = {f_xy(hist[-1]):.8e} en x = {hist[-1]}")
        plot_all(hist, rango, puntos)

run_button.on_click(on_run_clicked)

In [18]:
def on_iter_change(change):
    with output:
        if current_hist is None:
            return
        clear_output(wait=True)
        idx = change['new']
        rango = rango_w.value
        puntos = puntos_w.value
        plot_all(current_hist, rango, puntos, iter_index=idx)

iter_slider.observe(on_iter_change, names='value')

### Tips
- Para animar, mueve el slider de "Iter" lentamente o usa la reproducción del slider (en Jupyter lab a veces hay un botón play).
- Si la evaluación del Hessiano es lenta para mallas grandes, reduce `puntos` o aumenta `eps` en la hessiana numérica.
- Para exportar figura: clic derecho sobre la figura -> Save as...

### Guardado
Puedes guardar la trayectoria y resultados con:
```python
np.savetxt('historia_trayectoria.csv', current_hist, delimiter=',', header='x,y', comments='')

In [19]:
np.savetxt('historial/historia_trayectoria.csv', current_hist, delimiter=',', header='x,y', comments='')